<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Naas - Send notification

**Tags:** #naas #emailbuilder #snippet #operations

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook send a notification with your latest posts performance.

## Input

### Import libraries

In [1]:
from naas_drivers import emailbuilder, gsheet
import naas
import os
import naas_data_product
from datetime import date, datetime, timedelta
from naas_drivers import naasauth
import pandas as pd

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/openai.ipynb' successfully loaded.


### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_content`: Name of the file to be retrieved.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `datalake_dir`: Datalake directory to retrieve plugin json file.

**Outputs**
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email.
- `subject`: This variable is used for storing the subject of the notification email.

In [2]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())
file_content = "content"
input_image = "content_trend.png"
input_html = "content_trend.html"
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name = "CONTENT"
datalake_dir = naas_data_product.OUTPUTS_PATH

# Outputs
email_to = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "emails") or []
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())

## Model

### Set outputs

In [3]:
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "entity_name") or ""
timestamp_email = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
email_subject = f"📲 Content Assistant Report - {entity_name} ({timestamp_email})"
csv_output = os.path.join(output_dir, f"{datetime.now().strftime('%Y%m%d%H%M%S')}_{TW}_content-assistant-report_{entity_name.lower().replace(' ', '_')}.csv")
print("- CSV output:", csv_output)

- CSV output: /home/ftp/abi/outputs/content-engine/2023-12-26/20231226105610_W52-2023_content-assistant-report_jérémy_ravenel.csv


### Get assets

In [4]:
image_url = get_image_asset(input_dir, input_image)
print("- Image URL:", image_url)

image_html = get_image_asset(input_dir, input_html)
print("- Image HTML:", image_html)

plugin_url = naas.asset.find(os.path.join(datalake_dir, "plugins", "📲_content_assistant.json"))
print("- Plugin URL:", plugin_url)

- Image URL: https://public.naas.ai/amVyZW15LTQwbmFhcy0yRWFp/asset/a97c877b6d6e9a1ae983a133a90f115fdba739a43918fca3fc204110cf1e.png
- Image HTML: https://public.naas.ai/amVyZW15LTQwbmFhcy0yRWFp/asset/56adc1fab9ca40115970d3c8e3a1bf947b14ffe18a6795814d68aac421d1
- Plugin URL: https://public.naas.ai/amVyZW15LTQwbmFhcy0yRWFp/asset/6d5623845d24a2cc8f1262f627ef251ececcc7ab3407eacc408d99b1ad43


### Get data

In [9]:
df_input = pload(input_dir, file_content)    
if df_input is None:
    df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)

# Filter on the 2 weeks
df_input = df_input[df_input["SCENARIO"].isin([TW, LW])]

# Display result
df_input.to_csv(csv_output, index=False)
print("Rows:", len(df_input))
df_input.head(1)

Rows: 6


,ENTITY,SCENARIO,SOURCE,PUBLISHED_DATE,DATE,TIME,TITLE,CONTENT,TOPICS,CONTENT_LENGTH,KEYWORDS,VIEWS,LIKES,COMMENTS,SHARES,ENGAGEMENT_SCORE,CONTENT_URL
0,Matt Turck,W51-2023,LinkedIn,2023-12-23 22:43:09+0100,Sat. 23 Dec.,22H43,Thinking you're late to AI today is like think...,Thinking you're late to AI today is like think...,"""Artificial Intelligence, perception of latene...",85,None,0,0,0,0,0.0,https://www.linkedin.com/feed/update/urn:li:ac...


### Create Overview details

In [6]:
posts = []
df_posts = pd.DataFrame()
if len(df_input) > 0 and "SCENARIO" in df_input.columns:
    df_posts = df_input.copy()
    df_posts = df_posts[df_posts["SCENARIO"] == tw]
    
for row in df_posts.itertuples():
    title = row.TITLE
    url = row.CONTENT_URL
    content = row.TOPICS
    post_date = row.DATE
    post_time = row.TIME
    topic = row.TOPICS
    posts.append(f"<a href='{url}'>{title}</a> published on {post_date} at {post_time} on '{topic}'")
if len(posts) == 0:
    posts = [""]
print(posts)

['']


### Create 'Performance' details

In [7]:
# Groupby weeks
df = df_input.groupby(["SCENARIO"], as_index=False).agg({"VIEWS": "sum", "LIKES": "sum", "COMMENTS": "sum", "SHARES": "sum"})
df = df.sort_values(by="SCENARIO").reset_index(drop=True)

def create_var_text(df, column):
    # Get value this scenario and last scenario
    if len(df) == 0:
        value = 0
        value_n1 = 0
    elif len(df) == 1:
        if df.loc[0, "SCENARIO"] == tw:
            value = df.loc[0, column]
            value_n1 = 0
        else:
            value = 0
            value_n1 = df.loc[0, column]
    else:
        value = df.loc[df.index[-1], column]
        value_n1 = df.loc[df.index[0], column]
        
    # Calc variation in value
    varv = value - value_n1
    if varv < 0:
        message = f"{format_number(value)} (-{format_number(varv)})"
    elif varv == 0:
        message = f"{format_number(value)} (unchanged)"
    else:
        message = f"{format_number(value)} (+{format_number(varv)})"
    return message

views = create_var_text(df, "VIEWS")
likes = create_var_text(df, "LIKES")
comments = create_var_text(df, "COMMENTS")
shares = create_var_text(df, "SHARES")

# Preview of what will be send by email:
kpis = [
    f"Views : <b>{views}</b>",
    f"Likes : <b>{likes}</b>",
    f"Comments : <b>{comments}</b>",
    f"Shares : <b>{shares}</b>",
]
if len(kpis) == 0:
    kpis = [""]
print(kpis)

['Views : <b>0 (-25 302)</b>', 'Likes : <b>0 (-192)</b>', 'Comments : <b>0 (-58)</b>', 'Shares : <b>0 (-11)</b>']


### Generate email content

In [8]:
def generate_email_content(
    entity_name,
    image_url,
    image_html,
    posts,
    kpis,
    plugin_url,
):
    content = {
        'heading': "📲 Content Assistant",
        "txt_intro": (
            f"Hi {entity_name.split(' ')[0]},<br><br>" "I hope this email finds you well. Here is the latest update on your content performance."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="25px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(image_url, link=image_html),
        "text_1": emailbuilder.text(
            "Content published this week:"
        ),
        "list_1": emailbuilder.list(posts),
        "title_2": emailbuilder.text(
            "Performance", font_size="25px", text_align="center", bold=True
        ),
        "text_2": emailbuilder.text(
            "Statistics vs last week:"
        ),
        "list_2": emailbuilder.list(kpis),
        "text_3": emailbuilder.text(
            "I am here to support you in optimizing your content strategy and achieving your goals. If you would like to discuss any specific ideas or strategies, I am available for a chat at your convenience."
        ),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin_url}"),
            text="Start Chatting",
            background_color="#181a1c",
        ),
        "footer_naas": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = generate_email_content(
    entity_name,
    image_url,
    image_html,
    posts,
    kpis,
    plugin_url,
)

## Output

### Send the email

In [ ]:
if len(email_to) > 0:
    naas.notification.send(
        email_to=list(email_to.split(",")),
        subject=email_subject,
        html=email_content,
        files=[csv_output],
    )